<a href="https://colab.research.google.com/github/JerryChenz/Invest_Proc/blob/master/equity_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Set inputs

We load the sample dataset from the my github repository, and display the dataset in pandas.

In [53]:
import pandas as pd
import numpy as np
summary_url = 'https://raw.githubusercontent.com/JerryChenz/Invest_Proc_Open/main/financial_models/Opportunities/Screener/screener_summary_latest.csv'
df = pd.read_csv(summary_url)
# prepare the data
df.rename(columns={'Unnamed: 0': 'ticker'}, inplace=True)
# exclude minor countries with no forex quote on yahoo
df = df[df['fxRate'] != 0]
# rename the first column
# display(df)

# Step 2: Pre-screening

We can screen using different sets of conditions, then merge them later.

In [54]:
# Filters by Listing Location
market = 'HK' #@param ["HK","CN", "US", "Any"]

# Filters by Business Sector -- df.Sector.unique()
sector = "Any" #@param ["Real Estate", "Consumer Cyclical", "Industrials", "Energy", "Utilities", "Healthcare", "Basic Materials", "Financial Services", "Consumer Defensive", "Technology", "Communication Services", "Any"]

In [55]:
"""
Available variables:
'shortName', 'sector', 'industry', 'market', 'price', 'priceCurrency', 
'sharesOutstanding', 'reportCurrency', 'fxRate', 
'lastFiscalYearEnd', 'mostRecentQuarter', 'lastDividend', 'lastBuyback',
'totalAssets', 'currentAssets', 'currentLiabilities',
'totalAssets_-1', 'currentAssets_-1', 'currentLiabilities_-1',
'cashAndCashEquivalents', 'otherShortTermInvestments',
'cashAndCashEquivalents_-1', 'otherShortTermInvestments_-1',
'currentDebtAndCapitalLease', 'currentCapitalLease',
'currentDebtAndCapitalLease_-1', 'currentCapitalLease_-1',
'longTermDebtAndCapitalLease', 'longTermCapitalLease',
'longTermDebtAndCapitalLease_-1', 'longTermCapitalLease_-1',
'totalEquityAndMinorityInterest', 'commonStockEquity',
'totalEquityAndMinorityInterest_-1', 'commonStockEquity_-1',
'investmentProperties', 'longTermEquityInvestment', 'longTermFinancialAssets',
'investmentProperties_-1', 'longTermEquityInvestment_-1', 'longTermFinancialAssets_-1',
'netPPE', 'totalRevenue', 'costOfRevenue', 'sellingGeneralAndAdministration',
'netPPE_-1', 'totalRevenue_-1', 'costOfRevenue_-1', 'sellingGeneralAndAdministration_-1',
'netIncomeCommonStockholders', 'interestPaidCfo', 'interestPaidCff',
'netIncomeCommonStockholders_-1', 'interestPaidCfo_-1', 'interestPaidCff_-1',
'cfo', 'cfi', 'cff', 'endCashPosition',
'cfo_-1', 'cfi_-1', 'cff_-1', 'endCashPosition_-1'
"""

"""0. Definitions"""
# capitalization in reporting currency
capitalization_price = df['price'] * df['sharesOutstanding']
capitalization_report = capitalization_price / df['fxRate']
# dividend rate & buyback rate
dividend_rate = df['lastDividend'] / df['price']
buyback_rate = df['lastBuyback'] / df['price']
# Capital Structure related
total_debt = df['currentDebtAndCapitalLease'] + df['longTermDebtAndCapitalLease']
noncommonInterest = df['totalEquityAndMinorityInterest'] - df['commonStockEquity']
# Operating assets and liabilities
op_assets = df['totalAssets'] - df['cashAndCashEquivalents']
op_liabilities = df['totalAssets'] - df['totalEquityAndMinorityInterest'] - total_debt
net_op_assets = op_assets - op_liabilities

"""1. Stability Ratios"""
# leverage ratio
current_ratio = df['currentAssets'] / df['currentLiabilities']
debt_ratio = total_debt / df['commonStockEquity']
# Accrual anomaly - Scaled Total Accruals
# sta = (df['NetIncomeCommonStockholders'] - df['CFO']) / df['TotalAssets']
# Scaled Net Operating Assets
sona = net_op_assets / df['totalAssets']
# Todo: Dr. Messod Beneish's PROBM model

"""2. Quality Ratios: """
ppe_multiple = df['netPPE'] / df['totalRevenue']
# Net working Capital
net_working_capital = np.where((df['currentAssets'] == 0) & (df['currentLiabilities'] == 0), df['commonStockEquity'], df['currentAssets'] - df['currentLiabilities'])
# Return measurements
grossProfit = df['totalRevenue'] - df['costOfRevenue']
grossProfit_1 = df['totalRevenue_-1'] - df['costOfRevenue_-1']
ebit = grossProfit - df['sellingGeneralAndAdministration']
ebit_1 = grossProfit_1 - df['sellingGeneralAndAdministration_-1']
# ROC
greenblatt_capital = df['netPPE'] + net_working_capital - df['cashAndCashEquivalents']  # Not consider capital structure
greenblatt_roc = ebit / greenblatt_capital

"""Liquidity test"""
# liquidity_coverage_ratio
discounted_st_investment = df['otherShortTermInvestments']* 0.9
core_liquid_assets = df['cashAndCashEquivalents'] + discounted_st_investment
core_lcr = (core_liquid_assets) / df['currentLiabilities']
lcr = (core_liquid_assets + df['longTermFinancialAssets']) / df['currentLiabilities']

"""Price Ratios: """
# defining excess_cash
excess_cash = np.where(net_working_capital >= df['cashAndCashEquivalents'], df['cashAndCashEquivalents'], net_working_capital)
# Non-operating_noncash assets
nonop_noncash_assets = 0.9 * (df['investmentProperties'] + df['longTermEquityInvestment']) + discounted_st_investment 
# enterprise value
enterprise_value = capitalization_report + total_debt + noncommonInterest - excess_cash - nonop_noncash_assets
ebit_tev = ebit / enterprise_value

df['sales_growth'] = df['totalRevenue']/df['totalRevenue_-1']-1
df['Avg_Gross_margin'] = (grossProfit/df['totalRevenue'] + grossProfit_1/df['totalRevenue_-1'])/2
df['Avg_ebit_margin'] = (ebit/df['totalRevenue'] + ebit_1/df['totalRevenue_-1'])/2

df['EV'] = enterprise_value # may need to update the price
df['EBIT/EV'] = ebit_tev
df['ROC'] = greenblatt_roc
df['Debt ratio'] = debt_ratio
df['Current ratio'] = current_ratio
df['Dividend rate'] = dividend_rate
df['Buyback rate'] = buyback_rate
df['PPE_multiple'] = ppe_multiple
df['lcr'] = lcr
df['core_lcr'] = core_lcr

# display(df)

In [56]:
# filter by market
if market == 'HK':
  exchange_condition = df['market'] == 'hk_market'
elif market == 'CN':
  exchange_condition = df['market'] == 'cn_market'
elif market == 'US':
  exchange_condition = df['market'] == 'us_market'
else:
  exchange_condition = (df['market'] not in ['hk_market', 'cn_market', 'us_market'])

df = df.loc[exchange_condition]

# print(df.to_string())

In [57]:
# filter by sector
if sector == 'Any':
  sector_condition = df['sector'] != ''
else:
  if sector == "Real Estate":
    missing_stocks = df['ticker'].str.contains("0019.HK")
    not_service = ~df["shortName"].str.contains("SERVICES") & ~df["shortName"].str.contains("MOTOR") & ~df["shortName"].str.contains("SERVICE") & ~df["shortName"].str.contains("SER")
    sector_condition = ((df['sector'] == sector) | missing_stocks) & (not_service)
  else:
    sector_condition = df['sector'] == sector

df = df.loc[sector_condition]

# Unfiltered Data
display(df[['ticker', 'shortName', 'price', 'priceCurrency', 'Dividend rate', 'Buyback rate', 'sales_growth', 'Avg_Gross_margin', 'Avg_ebit_margin', 'ROC', 'lcr' ,'core_lcr', 'Current ratio', 'Debt ratio']])

,ticker,shortName,price,priceCurrency,Dividend rate,Buyback rate,sales_growth,Avg_Gross_margin,Avg_ebit_margin,ROC,lcr,core_lcr,Current ratio,Debt ratio
0,0001.HK,CKH HOLDINGS,49.250000,HKD,0.051037,0.006568,0.054246,0.523470,0.431255,1.107270,0.827404,0.741363,1.258465,0.693121
1,0002.HK,CLP HOLDINGS,57.700001,HKD,0.053726,0.000000,0.054894,0.334693,0.334693,0.189670,0.056880,0.056880,0.984785,0.565641
2,0003.HK,HK & CHINA GAS,7.720000,HKD,0.043918,0.000000,0.308762,0.427160,0.427160,0.417050,0.488018,0.291208,0.687538,0.880843
3,0004.HK,WHARF HOLDINGS,18.980000,HKD,0.021068,0.000000,0.065771,0.577077,0.489229,0.453281,0.474061,0.474061,1.683942,0.185752
4,0005.HK,HSBC HOLDINGS,58.200001,HKD,0.005492,0.004770,0.012102,1.000000,0.820553,-0.394794,inf,inf,NaN,0.492669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,9995.HK,REMEGEN-B,59.599998,HKD,0.000000,0.000000,inf,NaN,-inf,0.298722,5.494082,5.472693,7.098827,0.016168
2160,9996.HK,PEIJIA-B,12.300000,HKD,0.000000,0.000000,2.532117,0.676552,-1.883204,-0.441710,12.451431,11.066609,12.273423,0.023597
2161,9997.HK,KANGJI MEDICAL,9.560000,HKD,0.003855,0.004391,0.349514,0.829540,0.626402,3.263778,10.517410,10.151334,10.905773,0.014715
2162,9998.HK,KWAN YONG,0.157000,HKD,0.000000,0.000000,-0.141020,-0.004430,-0.085028,-0.129261,0.443159,0.414709,1.505319,0.138512


In [58]:
# common fitlering conditions: 
# 1. Reasonable leverage and Good Liqudity
common_1 = (core_lcr >= 0.2) & (lcr >= 0.6)
common_2 = (current_ratio >= 0.8) & (debt_ratio <= 0.8)
common_3 = (total_debt / df['currentAssets']) < 1.5
# 2. Avoid negative gross margin and value trap
common_4 = capitalization_price > 1000000000
common_5 = (capitalization_price > 6000000000) & (df['Avg_Gross_margin'] > 0.01)
common_6 = (capitalization_price <= 6000000000) & ((dividend_rate > 0.01) | (buyback_rate > 0.01))

# filtered by common conditions
common_df = df
common_df = common_df.loc[common_1 & common_2 & common_3 & common_4 & (common_5 | common_6)]
# display(common_df.loc[:, ['Ticker', 'Name', 'Price', 'Price_currency', 'Dividend rate', 'Buyback rate', 'Avg_sales_growth', 'Avg_Gross_margin', 'Avg_ebit_margin', 'ROC', 'lcr' ,'core_lcr', 'Current ratio', 'Debt ratio']])

#Step 3. Screening

Filter the dataset using the above conditions

In [59]:
# 1st set of conditions: Stability
asset_1 = (core_lcr >= 0.8) & (lcr >= 0.9)
# asset_2 = (dividend_rate > 0.01) | (buyback_rate > 0.01)
asset_3 = (current_ratio >= 1) & (debt_ratio <= 0.5)

# filtered by 1st set of conditions: Stability
asset_df = common_df
asset_df['is_Asset'] = True
asset_df = asset_df.loc[asset_1 & asset_3]
# display(asset_df.loc[:, ['Ticker', 'Name', 'Price', 'Price_currency', 'Dividend rate', 'Buyback rate', 'ROC', 'Avg_sales_growth', 'Avg_Gross_margin', 'Avg_ebit_margin', 'lcr' ,'core_lcr', 'Current ratio', 'Debt ratio']])

<ipython-input-59-93b09a36c3f4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_df['is_Asset'] = True


In [60]:
# 2nd set of conditions: Stalwart
stalwart_1 = (df['Avg_Gross_margin'] > 10) & (df['Avg_ebit_margin'] > 15)
stalwart_2 = df['sales_growth'] >= -0.10
stalwart_3 = greenblatt_roc > 0.03

# fitlered by 2nd set of conditions: Stalwart
Stalwart_df = common_df
Stalwart_df['is_Stalwart'] = True
Stalwart_df = Stalwart_df.loc[stalwart_1 & stalwart_2 & stalwart_3]
# display(Stalwart_df.loc[:, ['Ticker', 'Name', 'Price', 'Price_currency', 'Dividend rate', 'Buyback rate', 'Avg_sales_growth', 'Avg_Gross_margin', 'Avg_ebit_margin', 'ROC', 'lcr' ,'core_lcr', 'Current ratio', 'Debt ratio']])

<ipython-input-60-3fa03c7d768c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Stalwart_df['is_Stalwart'] = True


In [61]:
# combine the results
result_set = pd.concat([Stalwart_df, asset_df])

# Sort the set
result_set = result_set.sort_values('EV')

# find the subset of Asset_play and Stalwart
is_asset_stalwart = [result_set['is_Asset'].eq(True) & result_set['is_Stalwart'].eq(True)]
result_set['A_S'] = np.select(is_asset_stalwart, [True], default='')

display(result_set[['ticker', 'shortName', 'sector', 'price', 'Dividend rate', 'Buyback rate', 'EV', 'EBIT/EV', 'ROC', 'sales_growth', 'Avg_Gross_margin', 'Avg_ebit_margin', 'lcr' ,'core_lcr', 'Current ratio', 'Debt ratio', 'A_S']])

,ticker,shortName,sector,price,Dividend rate,Buyback rate,EV,EBIT/EV,ROC,sales_growth,Avg_Gross_margin,Avg_ebit_margin,lcr,core_lcr,Current ratio,Debt ratio,A_S
69,0083.HK,SINO LAND,Real Estate,10.460000,0.036215,0.000000,-2.887693e+10,-0.255734,0.136724,-0.366309,0.540982,0.496326,2.757202,2.691184,4.728685,0.032396,
833,1114.HK,BRILLIANCE CHI,Consumer Cyclical,3.710000,0.000000,0.000000,-2.254261e+10,0.128701,-1.342332,-0.314184,0.045164,-1.062477,4.455669,4.453261,5.871333,0.028294,
691,0914.HK,CONCH CEMENT,Basic Materials,28.900000,0.307057,0.000000,-1.763846e+10,-2.569035,0.526646,-0.047038,0.293979,0.265920,2.266614,2.143545,3.052501,0.112308,
667,0878.HK,SOUNDWILL HOLD,Real Estate,7.390000,0.027064,0.000000,-1.489151e+10,-0.022058,0.401553,-0.175867,0.854595,0.591933,1.260042,1.218220,2.482357,0.102041,
243,0289.HK,WING ON CO,Consumer Cyclical,16.040001,0.066241,0.001742,-1.305117e+10,-0.056574,0.437479,-0.049146,0.644726,0.644726,9.526092,9.526092,10.342363,0.002522,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,0027.HK,GALAXY ENT,Consumer Cyclical,53.599998,0.000000,0.000081,2.348630e+11,0.040575,0.254268,0.529619,0.571385,0.460281,1.830154,1.182146,1.306274,0.142414,
671,0883.HK,CNOOC,Energy,11.460000,0.040076,0.000000,2.922581e+11,0.549726,0.314743,0.573098,0.629814,0.583901,1.497702,1.446177,1.893241,0.236530,
2163,9999.HK,NTES-S,Communication Services,137.199997,0.007787,0.000031,3.171715e+11,0.096141,0.486105,0.189215,0.532671,0.343091,1.944385,1.865936,2.307475,0.275192,
2153,9988.HK,BABA-SW,Consumer Cyclical,102.500000,0.000000,0.028202,1.341876e+12,0.120645,0.604739,0.189286,0.390206,0.205960,1.199052,1.199052,1.770118,0.170307,


In [62]:
display_set = result_set.loc[result_set['A_S'] != '']
display(display_set[['ticker', 'shortName', 'sector', 'price', 'Dividend rate', 'Buyback rate', 'EV', 'EBIT/EV', 'ROC', 'sales_growth', 'Avg_Gross_margin', 'Avg_ebit_margin', 'lcr' ,'core_lcr', 'Current ratio', 'Debt ratio']])

,ticker,shortName,sector,price,Dividend rate,Buyback rate,EV,EBIT/EV,ROC,sales_growth,Avg_Gross_margin,Avg_ebit_margin,lcr,core_lcr,Current ratio,Debt ratio


In [63]:
# Export the screen result to csv file
export = 'No' #@param ["Yes","No"]
raw = 'No' #@param ["Yes","No"]
simplified = 'Yes' #@param ["Yes","No"]

if export == "Yes":
  from google.colab import files
  csv_name = "Screener_result.csv"
  if raw == "Yes":
    csv_name = "unfiltered_result.csv"
    if simplified == "Yes":
      df[['Ticker', 'Name', 'Sector', 'Price', 'Price_currency', 'Reporting_Currency', 'Dividend rate', 'Buyback rate', 'ROC', 'Avg_sales_growth', 'Avg_Gross_margin', 'Avg_ebit_margin', 'lcr' ,'core_lcr', 'Current ratio', 'Debt ratio']].to_csv(csv_name)
    else:
      csv_name = f"Raw_{csv_name}"
      df.to_csv(csv_name)
  else:
    if simplified == "Yes":
      result_set[['Ticker', 'Name', 'Sector', 'Price', 'Price_currency', 'Reporting_Currency', 'Dividend rate', 'Buyback rate', 'EV', 'EBIT/EV', 'ROC', 'Avg_sales_growth', 'Avg_Gross_margin', 'Avg_ebit_margin', 'lcr' ,'core_lcr', 'Current ratio', 'Debt ratio', 'A_S']].to_csv(csv_name)
    else:
      csv_name = f"Raw_{csv_name}"
      result_set.to_csv(csv_name)

  # export
  files.download(csv_name)

print("***End of the Screener***")

***End of the Screener***
